# Variational Autoencoder for Regression - WIP

- Paper https://arxiv.org/abs/1904.05948

- Repository https://github.com/QingyuZhao/VAE-for-Regression


In [11]:
import itertools
import numpy as np 
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils import weight_norm
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error as mse, r2_score 

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [13]:
device

device(type='cuda')

## Load Example Data

In [24]:
# Load Toy Example Data
training_feature = np.loadtxt('data/X.txt')
training_feature.shape

Y = np.loadtxt('data/Y.txt')
ground_truth_r = Y

np.random.seed(seed=0)

original_dim = training_feature.shape[1]
num_train = training_feature.shape[0]

In [48]:
def sampling(args):
    '''
    Arguments:
        args (tensor): mean and log of variance of Q(z|X)
    Returns:
        z (tensor): sampled latent vector
    '''
    mean, log_var = args
    epsilon = torch.randn_like(torch.Tensor(mean)) 
    return mean + torch.exp(0.5*log_var)*epsilon 

## Build VAE Regression Model 

In [49]:
class Encoder(nn.Module):
    def __init__(self, input_shape_x, intermediate_dim, latent_dim):
        super(Encoder, self).__init__()
        self.dropout = nn.Dropout(p=0.25)
        self.fc1 = nn.Linear(input_shape_x, 128)
        self.act1 = nn.Tanh()
        self.fc2 = nn.Linear(128, intermediate_dim)
        self.act2 = nn.Tanh() 
        
        # posterior on Y; probabilistic regressor 
        self.r_mean = nn.Linear(intermediate_dim, 1)
        self.r_logvar = nn.Linear(intermediate_dim, 1) 

        # q(z|x) 
        self.z_mean = nn.Linear(intermediate_dim, latent_dim)
        self.z_logvar = nn.Linear(intermediate_dim, latent_dim)

        # latent generator 
        self.gen_z = weight_norm(nn.Linear(1, latent_dim))


    def forward(self, x):
        x = self.dropout(x)
        x = self.act1(self.fc1(x))
        x = self.act2(self.fc2(x))

        r_mean = self.r_mean(x)
        r_logvar = self.r_logvar(x)

        z_mean = self.z_mean(x)
        z_logvar = self.z_logvar(x)

        # reparameterization trick
        r = sampling([self.r_mean, self.r_logvar])
        z = sampling([self.z_mean, self.z_logvar])

        pz_mean = self.gen_z(r) 

        return r_mean, r_logvar, r, z_mean, z_logvar, z, pz_mean


In [50]:
class Decoder(nn.Module):
    def __init__(self, input_shape_x, intermediate_dim, latent_dim):
        super(Decoder, self).__init__()
        self.fc1 = nn.Linear(latent_dim, intermediate_dim)
        self.act1 = nn.Tanh()
        self.fc2 = nn.Linear(intermediate_dim, 128)
        self.act2 = nn.Tanh()
        self.fc3 = nn.Linear(128, input_shape_x)

    
    def forward(self, x):
        x = self.act1(self.fc1(x))
        x = self.act2(self.fc2(x))
        x = self.fc3(x)
        return x 

In [51]:
encoder = Encoder(original_dim, 32, 8).to(device)
decoder = Decoder(original_dim, 32, 8).to(device) 

## Dataset and DataLoader 

In [52]:
class VAEDataset(Dataset):
    def __init__(self, images, labels):
        self.images = torch.Tensor(images)
        self.labels = torch.Tensor(labels) 
    def __len__(self):
        return len(self.images)
    def __getitem__(self, idx):
        imgs = self.images[idx]
        lbls = self.labels[idx]
        return imgs, lbls 

## Hyperparameters

In [53]:

intermidiate_dim = 32
batch_size = 64
latent_dim = 8
epochs = 100
lr = 0.001
mse = nn.MSELoss()

## Loss Function

In [54]:
# reconstruction_loss = mse(inputs_x,outputs)
# #kl_loss = 1 + z_log_var - pz_log_var - K.tf.divide(K.square(z_mean-pz_mean),K.exp(pz_log_var)) - K.tf.divide(K.exp(z_log_var),K.exp(pz_log_var))
# kl_loss = 1 + z_log_var - K.square(z_mean-pz_mean) - K.exp(z_log_var)
# kl_loss = -0.5*K.sum(kl_loss, axis=-1)
# label_loss = K.tf.divide(0.5*K.square(r_mean - inputs_r), K.exp(r_log_var)) +  0.5 * r_log_var

# vae_loss = K.mean(reconstruction_loss+kl_loss+label_loss)


In [55]:
def loss_function(inputs_x, outputs, inputs_r, z_mean, z_log_var, r_mean, r_logvar, pz_mean):
    reconstruction_loss = mse(inputs_x,outputs)
    kl_loss = 1 + z_logvar - torch.square(z_mean-pz_mean) - torch.exp(z_logvar)
    kl_loss = -0.5 * torch.sum(kl_loss) 
    label_loss = torch.divide(0.5 * torch.square(r_mean - inputs_r), troch.exp(r_logvar)) +  0.5 * r_logvar
    return torch.mean(reconstruction_loss+kl_loss+label_loss)

In [56]:
# # KL divergence
# def loss_fn(out, imgs, mu, logVar):
#     kl_divergence = 0.5 * torch.sum(1 + logVar - mu.pow(2) - logVar.exp())
#     return F.binary_cross_entropy(out, imgs, size_average=False) - kl_divergence

lr= 0.001 # Learning rate

params_to_optimize = [
    {'params': encoder.parameters()},
    {'params': decoder.parameters()}
]

optim = torch.optim.Adam(params_to_optimize, lr=lr)

### Optmizer 

In [57]:
#ground_truth_r
#training_feature_sk

In [59]:
# ## Train the network
# np.random.seed(0)
# skf = StratifiedKFold(n_splits=10)
# pred = np.zeros((ground_truth_r.shape))
# fake = np.zeros((ground_truth_r.shape[0]))
# fake[:300] = 1


# # Run 10-fold CV
# for train_idx, test_idx in skf.split(training_feature, fake):
#     training_feature_sk = training_feature[train_idx,:]
#     training_score = ground_truth_r[train_idx]
#     testing_feature_sk = training_feature[test_idx,:]
#     testing_score = ground_truth_r[test_idx]    
    
#     train = VAEDataset(training_feature_sk, training_score)
#     test = VAEDataset(testing_feature_sk, testing_score)
#     train_dataloader = DataLoader(train, batch_size=batch_size, shuffle=True) 
#     test_dataloader = DataLoader(test, batch_size=batch_size, shuffle=False)     
    
#     for epoch in range(epochs):
        
#         encoder.train()
#         decoder.train() 
        
#         for idx, data in enumerate(train_dataloader):
        
#             imgs, lbls = data
#             imgs = imgs.to(device)
#             lbls = lbls.to(device)
            
#             r_mean, r_logvar, r, z_mean, z_logvar, z, pz_mean = encoder(imgs)
            
#             vae_loss = loss_function(inputs_x, outputs, lbls, r_mean, r_logvar, z_mean, z_log_var, pz_mean)
            
            
#             optmizer.zero_grad()
            
#             vae_loss.backward()
            
#             optmizer.step()
            
            
#     print(test_idx)
            

### Validation

In [ ]:
# # Mean squared error
# print("Mean squared error: %.3f" % mean_squared_error(ground_truth_r, pred))
# # Explained variance score: 1 is perfect prediction
# print('R2 Variance score: %.3f' % r2_score(ground_truth_r, pred))

# # Plot Prediction vs. Ground-truth Y
# fig = plt.figure()
# ax = fig.add_subplot(111)

# ax.scatter(ground_truth_r, pred,  color='black')
# plt.xlabel('ground truth')
# plt.ylabel('prediction truth')
# ax.axis('equal');

### Visualize Latent Space


In [ ]:
vae.load_weights('random_weights.h5')
vae.fit([training_feature,ground_truth_r],
         epochs=epochs,
         batch_size=batch_size,
         verbose = 0)
 
[z_mean, z_log_var, z, r_mean, r_log_var, r_vae, pz_mean] = encoder.predict([training_feature,ground_truth_r],batch_size=batch_size)

tsne = MDS(n_components=2, random_state=0)
X_2d = tsne.fit_transform(z_mean)

#%matplotlib notebook
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(X_2d[:, 0], X_2d[:, 1], c=ground_truth_r)
plt.title('TSNE visualization of latent space')
ax.axis('equal')

## Train the network
np.random.seed(0)
skf = StratifiedKFold(n_splits=10)
pred = np.zeros((ground_truth_r.shape))
fake = np.zeros((ground_truth_r.shape[0]))
fake[:300] = 1
